In [5]:
import sys
sys.path.append("../")

from transforms import *
import yaml
from types import SimpleNamespace

In [28]:
CONFIG_PATH = "../config.yaml"

with open(CONFIG_PATH) as file:
  config = SimpleNamespace(**yaml.safe_load(file))
  
  print(config)

namespace(dataset={'train_dir': './data/raw/IRMAS_Training_Data', 'valid_dir': './data/raw/IRMAS_Validation_Data', 'preprocess': 'PreprocessPipeline', 'transforms': 'FeatureExtractor', 'augments': False, 'SAMPLING_RATE': 16000, 'BATCH_SIZE': 32}, loss={'name': 'FocalLoss', 'args': {'pos_weight': 1}}, optimizer={'name': 'Adam', 'args': {'lr': 0.1, 'weight_decay': 0.01}}, verbose=True, EPOCHS=100, metrics=['hamming_score', 'zero_one_score', 'mean_f1_score', 'per_instr_f1_score'], model='ASTModel', scheduler={'name': 'ReduceLROnPlateau', 'args': {'patience': 2, 'threshold': 0.001}}, early_stopping={'enable': True, 'args': {'patience': 10, 'min_delta': 0.001}}, save_best_model=True)


In [3]:
config.metrics

['hamming_score', 'zero_one_score', 'mean_f1_score', 'per_instr_f1_score']

In [16]:
import torch.optim as optim

In [15]:
from utils import init_obj

In [20]:
import loss

In [22]:
import models

In [24]:
model = models.RNN(128, 64, 3, 11)

In [30]:
opt = init_obj(config.optimizer, optim, model.parameters())